<a href="https://colab.research.google.com/github/marcosvnespolo/Previsao_Vendas/blob/main/Previs%C3%A3o_de_Vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importando as Bibliotecas Necessárias

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import lightgbm as lgb


# 2. Carregando os Datasets Utilizados no Problema

In [22]:
features = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/features.csv')
stores = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/stores.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/test.csv')
train = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/train.csv')

# 3. Visualizando os Datasets

In [23]:
display(features.head())
print()
print()
display(stores.head())
print()
print()
display(test.head())
print()
print()
display(train.head())

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


# 4. Juntando os Datasets

É importante relacionar os datasets porque todos eles possuem imformações relevantes que se complementam entre sí

In [24]:
train = pd.merge(train, features, on=['Store', 'Date', 'IsHoliday'], how='left')
train = pd.merge(train, stores, on=['Store'], how='left')
test = pd.merge(test, features, on=['Store', 'Date', 'IsHoliday'], how='left')
test = pd.merge(test, stores, on=['Store'], how='left')

# 5. Limpeza e Pré-Processamento dos dados

In [27]:
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
le = LabelEncoder()
train['Type'] = le.fit_transform(train['Type'])
test['Type'] = le.transform(test['Type'])
train['IsHoliday'] = train['IsHoliday'].astype(int)
test['IsHoliday'] = test['IsHoliday'].astype(int)